In [1]:
import twitter
import ssl

from foodbornenyc.db_settings import twitter_config

api = twitter.Api(consumer_key=twitter_config['consumer_key'],
                  consumer_secret=twitter_config['consumer_secret'],
                  access_token_key=twitter_config['access_token'],
                  access_token_secret=twitter_config['access_token_secret'])

#print api.VerifyCredentials()


ImportError: No module named twitter

In [2]:
print twitter.status.Status.__doc__

A class representing the Status structure used by the twitter API.

  The Status structure exposes the following properties:

    status.contributors
    status.coordinates
    status.created_at
    status.created_at_in_seconds # read only
    status.favorited
    status.favorite_count
    status.geo
    status.id
    status.id_str
    status.in_reply_to_screen_name
    status.in_reply_to_user_id
    status.in_reply_to_status_id
    status.lang
    status.place
    status.retweet_count
    status.relative_created_at # read only
    status.source
    status.text
    status.truncated
    status.location
    status.user
    status.urls
    status.user_mentions
    status.hashtags
  


In [3]:
print twitter.user.User.__doc__

A class representing the User structure used by the twitter API.

  The User structure exposes the following properties:

    user.id
    user.name
    user.screen_name
    user.location
    user.description
    user.default_profile
    user.default_profile_image
    user.profile_image_url
    user.profile_background_tile
    user.profile_background_image_url
    user.profile_banner_url
    user.profile_sidebar_fill_color
    user.profile_background_color
    user.profile_link_color
    user.profile_text_color
    user.protected
    user.utc_offset
    user.time_zone
    user.url
    user.status
    user.statuses_count
    user.followers_count
    user.friends_count
    user.favourites_count
    user.geo_enabled
    user.verified
    user.lang
    user.notifications
    user.contributors_enabled
    user.created_at
    user.listed_count
  


In [16]:
print api.GetSearch.__doc__

Return twitter search results for a given term.

    Args:
      term:
        Term to search by. Optional if you include geocode.
      since_id:
        Returns results with an ID greater than (that is, more recent
        than) the specified ID. There are limits to the number of
        Tweets which can be accessed through the API. If the limit of
        Tweets has occurred since the since_id, the since_id will be
        forced to the oldest ID available. [Optional]
      max_id:
        Returns only statuses with an ID less than (that is, older
        than) or equal to the specified ID. [Optional]
      until:
        Returns tweets generated before the given date. Date should be
        formatted as YYYY-MM-DD. [Optional]
      geocode:
        Geolocation information in the form (latitude, longitude, radius)
        [Optional]
      count:
        Number of results to return.  Default is 15 [Optional]
      lang:
        Language for results as ISO 639-1 code.  Default is None

In [18]:
#for status in api.GetUserTimeline(screen_name="tomeffland"):
#    print dict(status)

In [224]:
retrieved_tweets = set()

In [241]:
search_terms = ['food poisoning',
                '#foodpoisioning',
                'vomit',
                '#vomit',
                'diarrhea',
                '#diarrhea',
                'puke',
                '#puke',
                'illness',
                '#illness',
                'stomach ache',
                '#stomachache',
                'the runs',
                '#theruns',
                'got sick'
                ]
terms = ' OR '.join(search_terms)
print terms
search_kwargs = {
      #'term':terms,
      #since_id:
      #  Returns results with an ID greater than (that is, more recent
      #  than) the specified ID. There are limits to the number of
      #  Tweets which can be accessed through the API. If the limit of
      #  Tweets has occurred since the since_id, the since_id will be
      #  forced to the oldest ID available. [Optional]
      #max_id:
      #  Returns only statuses with an ID less than (that is, older
      #  than) or equal to the specified ID. [Optional]
      #until:
      #  Returns tweets generated before the given date. Date should be
      #  formatted as YYYY-MM-DD. [Optional]
      'geocode':(40.7142700,-74.0059700, '100mi'),
      'count':100, #not sure what this max is
      'lang':'en',
      'result_type':'popular', #'popular' or 'mixed'
      'include_entities':True
    }

food poisoning OR #foodpoisioning OR vomit OR #vomit OR diarrhea OR #diarrhea OR puke OR #puke OR illness OR #illness OR stomach ache OR #stomachache OR the runs OR #theruns OR got sick


In [242]:
search = api.GetSearch(**search_kwargs)

In [243]:
old_ids = set(status.id for status in retrieved_tweets)
new_tweets = set(tweet for tweet in search if tweet.id not in old_ids)
retrieved_tweets |= new_tweets # union
new = len(retrieved_tweets) - old
print "Added %i new tweets" % new

Added 29 new tweets


In [244]:
for status in retrieved_tweets:
    mentions = [user.screen_name for user in status.user_mentions]
    print "Tweet: %s" % status.text.encode('utf-8')
    print "  By: @%s, Mentions:%s, in_reply_to: %s" % (status.user.screen_name, mentions, status.in_reply_to_screen_name)
    print "  Tweet id: %s" % status.id
    print

Tweet: "@AshlieJuliard: @KarlRove @realDonaldTrump Trump is the only candidate that I've ever been excited about #SomethingSpecialIsHappening"
  By: @realDonaldTrump, Mentions:[u'AshlieJuliard', u'KarlRove', u'realDonaldTrump'], in_reply_to: None
  Tweet id: 656638895388565506

Tweet: "@USATrustTrump: WOW! The More Opponents Push The Higher The #Trump2016 Reach! Keep Pushing DEMS And Jealous GOPs! https://t.co/RaS4FJAsls"
  By: @realDonaldTrump, Mentions:[u'USATrustTrump'], in_reply_to: None
  Tweet id: 657038182589054976

Tweet: "@ansun_sayavong: Your opponents have no idea what they're dealing with. Sun Tzu would be very proud. You are a tactical genius. #artofwar"
  By: @realDonaldTrump, Mentions:[u'ansun_sayavong'], in_reply_to: None
  Tweet id: 656669360208150528

Tweet: @TimHortons I ate one of your BLTs and got sick with food poisoning the next day. #nothappy.
  By: @sebrina_bender, Mentions:[u'TimHortons'], in_reply_to: TimHortons
  Tweet id: 654727113375772672

Tweet: Lost 15 

In [245]:
timeline_kwargs = {
    'screen_name':'happyenglishny',#None,
    'since_id':656330452756754432,#None,
    'count':100,
    'exclude_replies':False
}

In [76]:
# test out get timeline of a user who had a tweet in the response
timeline = api.GetUserTimeline(**timeline_kwargs)

In [179]:
def getRepliesToStatus(status):
    """Note: this is not provided by the API, so we are only get what search decides to give us"""
    user = status.user.screen_name
    potentials = api.GetSearch(term="@"+user,
                               since_id=status.id,
                               count=100,
                               lang='en',
                               include_entities=True
                              )
#     for status in potentials:
#         print "Tweet: %s" % status.text
#         print "  By: @%s, Mentions:%s, in_reply_to: %s" % (status.user.screen_name, mentions, status.in_reply_to_screen_name)
#         print "  Tweet id: %s" % status.id
#         print
    replies = [tweet for tweet in potentials if tweet.in_reply_to_status_id==status.id ]
    #print replies
    return replies

In [180]:
#teststatus = filter(lambda s:s.user.screen_name=='happyenglishny', retrieved_tweets)[0]
#replies = getRepliesToStatus(teststatus)

In [182]:
for status in retrieved_tweets:
    print
    print "Status by %s: %s" % (status.user.screen_name, status.text)
    for reply in getRepliesToStatus(status):
        #mentions = [user.screen_name for user in reply.user_mentions]
        print "Tweet: %s" % reply.text
        print "  By: @%s, in_reply_to: %s" % (reply.user.screen_name, reply.in_reply_to_screen_name)
        print "  Tweet id: %s" % reply.id
        print


Status by ripalwys: RT @happyenglishny: Eating food from the street vendor always gives me the runs (diarrhea) What gives you the runs?   #English #twinglish

Status by PartyGurl_Mason: In the past week: I was sick then got my bf sick…now I'm sick AGAIN because of food poisoning😒

Status by canu_diggit: Thank God they did because ALL of the people that went got sick as hell. Food poisoning. All of them

Status by Poorjim6060: @gardeneraly it was shocking!!!! Tina my wife got food poisoning 2 days in and I'm resisting the urge to be sick all the time......
Tweet: @Poorjim6060 😩 ok it was really awful.... Poor you :-/
  By: @gardeneraly, in_reply_to: Poorjim6060
  Tweet id: 655979718852911104


Status by iam_Dasiaa: This the second night i done got sick to the stomach and can't eat my food 😑😕

Status by sebrina_bender: @TimHortons I ate one of your BLTs and got sick with food poisoning the next day. #nothappy.
Tweet: @sebrina_bender This concerns us, pls follow back and DM with the exac

In [256]:
### THIS IS NOT ROBUST ###
# recursive conversation tracker
def recursiveConvoTree(status, paths, path):
    replies = getRepliesToStatus(status)
    if not replies:
        paths.append(path)
        return
    else:
        for reply in replies:
            path.append(reply)
            recursiveConvoTree(reply, paths, path)
    
# memoization wrapper of recursive algo
def trackConversations(root_status):
    paths = []
    path = [root_status]
    recursiveConvoTree(root_status, paths, path)
    return paths
    
def printConvoTree(convo_tree):
    if max([len(path) for path in convo_tree]) <=1 :
        print "No conversation occured"
    else:
        for i, path in enumerate(convo_tree):
            for j, t in enumerate(path):
                replyname = t.in_reply_to_screen_name
                if not replyname: replyname='Not in reply'
                print j*'  ' + t.text
                print j*'  ' + t.user.screen_name+' , '+replyname

In [234]:
print api.GetRateLimitStatus()[u'resources'][u'search']#[u'/search/tweets']

{u'/search/tweets': {u'reset': 1445486107, u'limit': 180, u'remaining': 154}}


In [218]:
# test recursive convo tree
teststatus = filter(lambda s:s.user.screen_name=='sebrina_bender', retrieved_tweets)[0]
convo_tree = trackConversations(teststatus)

In [257]:
for tweet in list(retrieved_tweets):
    convo_tree = trackConversations(tweet)
    print 
    printConvoTree(convo_tree)


No conversation occured

"@USATrustTrump: WOW! The More Opponents Push The Higher The #Trump2016 Reach! Keep Pushing DEMS And Jealous GOPs! https://t.co/RaS4FJAsls"
realDonaldTrump , Not in reply
  @realDonaldTrump @USATrustTrump Down with corrupt career politicians! Gooooo #TeamTrump16 #TrumpTrain #HispanicsForTrump #DemocratsForTrump
  TejadaDennes , realDonaldTrump

No conversation occured

@TimHortons I ate one of your BLTs and got sick with food poisoning the next day. #nothappy.
sebrina_bender , TimHortons
  @sebrina_bender This concerns us, pls follow back and DM with the exact location details where you visited and your phone number.
  TimHortons , sebrina_bender

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

No conversation occured

@WelloutonHES if he got sick last second wi

In [200]:
print convo_tree

[]


In [5]:
# testing with incident data
import json
incidents = []
with open('../stream_incidents2.json', 'r') as f:
    for line in f.readlines():
        incidents.append(json.loads(line))

In [6]:
print len(incidents)

110


In [20]:
inc = incidents[107]
tid = inc['tweet0']
convo =  inc['conversations'][0]
print inc['user']['screen_name']
for tweet in convo:
    print tweet['text']
    

DannyRatcliffe
I got food poisoning from eating at @Eataly's La Birreria last night. https://t.co/qF5SVyPa1X


In [23]:
for status in inc['timeline']:
    print status['text']

What's up, doc? #mood https://t.co/7W3Jyl5Tzf
@TheBrandyHoward you win at life. I can't handle how cute Mighty looks with bows! 🎀 #ThePeoplesCouch https://t.co/Jur2KxQSsh
Goodbye... It's you https://t.co/SJD58ITykb
RT @PersianLa27: In case you failed to notice, I could care less about your irrelevant ass.
"It's raining, it's pouring. The old man is snoring" is more meaningful now that I'm older.
@ercthms ?¿ https://t.co/JWrbirWGpD
@jjthornhill I have yet to ride to  Ariana's new song... I'm counting on you, so please Focus! 😋 https://t.co/wTHcS7xffj
RT @SartoriallyInc: where do u see yourself in 5 years? https://t.co/TthG3TiSY1
@jjthornhill @cycfitness uhhh DUH, #hairography is to be expected! https://t.co/EMequuzTSZ
I'm at @CycFitness in New York, NY https://t.co/Zll2vMXYsz
I hate that I love this. https://t.co/SWz7gxbAAg
RT @AlexPlaskey: DO NOT SEND ME A SNAPCHAT AND THEN PUT IT IN YOUR STORY. how dare you.
Two more classes to go! (at @CycFitness in New York, NY) https://t.co/OYlhMQo